### Importing the necessary modules

In [1]:
import pandas as pd
import random

### Filtering the Dataset

Filtering the shifts dataset to only include "Semantic Evolution" and "Microevolution"

In [2]:
data = pd.read_csv('DatSemShift.csv')

SC_data = data[(data['Type'] == ' Semantic evolution') | (data['Type'] == ' Microevolution')]

### Fixing the Direction Problem in the Dataset

Fixing the problems with the direction of the shifts in the dataset. Some of the shifts in the dataset are placed in reverse order (first the latest language is placed then the language that has the original lexeme). In these shifts languages, lexemes and meanings are swapped. Other shifts do not contain the direction of the change even though it can be understood from the chronology of the languages (e.g. if a shift have placed between Latin and Italian, it has to be from Latin to Italian since Latin precedes Italian). These are fixed by adding directions according to the chronologies.

In [3]:
for index, row in SC_data.iterrows():
    #Going through every row in the dataset
    if row['Direction'] == '←':
        #For the shifts where the direction is opposite, Language, Lexeme and Meaning 1 and 2 columns are replaced.
        SC_data.loc[index, 'Language_1'], SC_data.loc[index, 'Language_2'] = SC_data.loc[index, 'Language_2'], SC_data.loc[index, 'Language_1']
        SC_data.loc[index, 'Lexeme_1'], SC_data.loc[index, 'Lexeme_2'] = SC_data.loc[index, 'Lexeme_2'],  SC_data.loc[index, 'Lexeme_1']
        SC_data.loc[index, 'Meaning_1'], SC_data.loc[index, 'Meaning_2'] = SC_data.loc[index, 'Meaning_2'], SC_data.loc[index, 'Meaning_1']
        SC_data.loc[index, 'Direction'] = '→' #The direction is fixed

In [4]:
for index, row in SC_data.iterrows():
    #Going through each row of the dataset to find shifts that do not have direction
    if row['Direction'] != '→':
        if row['Language_2'] == 'Old ' + row['Language_1']:
            #If the old version of the languages is put as the second language, swapped the languages and place correct direction
            SC_data.loc[index, 'Language_1'], SC_data.loc[index, 'Language_2'] = SC_data.loc[index, 'Language_2'], SC_data.loc[index, 'Language_1']
            SC_data.loc[index, 'Lexeme_1'], SC_data.loc[index, 'Lexeme_2'] = SC_data.loc[index, 'Lexeme_2'],  SC_data.loc[index, 'Lexeme_1']
            SC_data.loc[index, 'Meaning_1'], SC_data.loc[index, 'Meaning_2'] = SC_data.loc[index, 'Meaning_2'], SC_data.loc[index, 'Meaning_1']
            SC_data.loc[index, 'Direction'] = '→'
        elif row['Language_1'] == 'Old ' + row['Language_2']:
            #If the original language is the old version of the second language
            SC_data.loc[index, 'Direction'] = '→'
        elif row['Language_1'] == 'Latin':
            #If first language is Latin and second language is one of the languages that derived from Latin
            if row['Language_2'] in ['French', 'Italian', 'Spanish', 'Catalan', 'Romanian', 'Arromanian', 'Romansh', 'Occitan', 'Galician']:
                SC_data.loc[index, 'Direction'] = '→'
        elif row['Language_1'] == 'Old Church Slavonic':
            #Old Church Slavonic precedes Bulgarian
            if row['Language_2'] == 'Bulgarian':
                SC_data.loc[index, 'Direction'] = '→'
        elif row['Language_1'] == 'Old East Slavic':
            #Old East Slavic precedes Russian
            if row['Language_2'] == 'Russian':
                SC_data.loc[index, 'Direction'] = '→'
        elif row['Language_1'] == row['Language_2'].replace('Modern', 'Ancient'):
            #If first language is the ancient and second language is the modern version of the same language 
            SC_data.loc[index, 'Direction'] = '→'

### Merging the Shifts Data

Merging the shifts that have the same shift id into the same row. The code takes the csv that includes all of the shifts, and groups the shifts that share a shift ID together and merges their remaining columns into the same row. 

In [5]:
#Keeping only the necessary columns in the dataframe
SC_data = SC_data[['ID', 'Type', 'Language_1', 'Lexeme_1', 'Meaning_1', 'Direction', 'Language_2', 'Lexeme_2', 'Meaning_2']]

#First, grouping the rows that have the same value in ID, Language 1, Lexeme 1 and Meaning 1 columns, and merging the Language 2 and 
#Lexeme 2 columns into the same row, every value separated by commas.
merged = SC_data.groupby(['ID', 'Language_1', 'Lexeme_1', 'Meaning_1']).agg({
    'Language_2': lambda x: ','.join(x.unique()),
    'Lexeme_2':   lambda x: ','.join(x.unique()),
    'Meaning_2':  'first',   
    'Type':       'first',   
    'Direction':  'first'
}).reset_index()
merged

,ID,Language_1,Lexeme_1,Meaning_1,Language_2,Lexeme_2,Meaning_2,Type,Direction
0,shift0007,Ancient Greek,μετατρέπω,to turn back,Modern Greek,μετατρέπω,to transform,Semantic evolution,→
1,shift0007,Irish Gaelic,iompaigh,"to turn, rotate (intr.)",Irish Gaelic,iompaigh,to become,Semantic evolution,→
2,shift0008,Irish Gaelic,eolas,to know,Irish Gaelic,eolas,to know how,Semantic evolution,→
3,shift0013,Irish Gaelic,comhair,"to calculate, count",Irish Gaelic,comhair,to take into account,Semantic evolution,→
4,shift0016,Irish Gaelic,milis,sweet (taste),Irish Gaelic,milis,"dear, darling",Semantic evolution,→
...,...,...,...,...,...,...,...,...,...
289,shift4646,Welsh,drych,to see/to look at,Welsh,drych,mirror,Semantic evolution,→
290,shift4647,Portuguese,dona,"female owner, mistress",Brazilian Portuguese,dona,wife,Semantic evolution,→
291,shift4663,Latin,libet (impers.),to want,Medieval Latin,quodlibet,anything,Semantic evolution,→
292,shift4720,Old East Slavic,дрѣмѫчии,sleeping,Old East Slavic,дремучий,dense (forest),Semantic evolution,→


In [6]:
#To not to lose any info related to the realizations of the shifts, encoding info into the same row
merged['Lexemes_1'] = merged['Language_1'] + ': [' + merged['Lexeme_1'] + ']'
merged['Meanings_1'] = merged['Lexeme_1'] + ': [' + merged['Meaning_1'] + ']'
merged['Languages_2'] = merged['Language_1'] + ': [' + merged['Language_2'] + ']'
merged['Lexemes_2'] = merged['Lexeme_1'] + ': [' + merged['Lexeme_2'] + ']'
merged['Meanings_2'] = merged['Lexeme_2'] + ': [' + merged['Meaning_2'] + ']'
merged

,ID,Language_1,Lexeme_1,Meaning_1,Language_2,Lexeme_2,Meaning_2,Type,Direction,Lexemes_1,Meanings_1,Languages_2,Lexemes_2,Meanings_2
0,shift0007,Ancient Greek,μετατρέπω,to turn back,Modern Greek,μετατρέπω,to transform,Semantic evolution,→,Ancient Greek: [μετατρέπω],μετατρέπω: [to turn back],Ancient Greek: [Modern Greek],μετατρέπω: [μετατρέπω],μετατρέπω: [to transform]
1,shift0007,Irish Gaelic,iompaigh,"to turn, rotate (intr.)",Irish Gaelic,iompaigh,to become,Semantic evolution,→,Irish Gaelic: [iompaigh],"iompaigh: [to turn, rotate (intr.)]",Irish Gaelic: [Irish Gaelic],iompaigh: [iompaigh],iompaigh: [to become]
2,shift0008,Irish Gaelic,eolas,to know,Irish Gaelic,eolas,to know how,Semantic evolution,→,Irish Gaelic: [eolas],eolas: [to know],Irish Gaelic: [Irish Gaelic],eolas: [eolas],eolas: [to know how]
3,shift0013,Irish Gaelic,comhair,"to calculate, count",Irish Gaelic,comhair,to take into account,Semantic evolution,→,Irish Gaelic: [comhair],"comhair: [to calculate, count]",Irish Gaelic: [Irish Gaelic],comhair: [comhair],comhair: [to take into account]
4,shift0016,Irish Gaelic,milis,sweet (taste),Irish Gaelic,milis,"dear, darling",Semantic evolution,→,Irish Gaelic: [milis],milis: [sweet (taste)],Irish Gaelic: [Irish Gaelic],milis: [milis],"milis: [dear, darling]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,shift4646,Welsh,drych,to see/to look at,Welsh,drych,mirror,Semantic evolution,→,Welsh: [drych],drych: [to see/to look at],Welsh: [Welsh],drych: [drych],drych: [mirror]
290,shift4647,Portuguese,dona,"female owner, mistress",Brazilian Portuguese,dona,wife,Semantic evolution,→,Portuguese: [dona],"dona: [female owner, mistress]",Portuguese: [Brazilian Portuguese],dona: [dona],dona: [wife]
291,shift4663,Latin,libet (impers.),to want,Medieval Latin,quodlibet,anything,Semantic evolution,→,Latin: [libet (impers.)],libet (impers.): [to want],Latin: [Medieval Latin],libet (impers.): [quodlibet],quodlibet: [anything]
292,shift4720,Old East Slavic,дрѣмѫчии,sleeping,Old East Slavic,дремучий,dense (forest),Semantic evolution,→,Old East Slavic: [дрѣмѫчии],дрѣмѫчии: [sleeping],Old East Slavic: [Old East Slavic],дрѣмѫчии: [дремучий],дремучий: [dense (forest)]


In [7]:
#Merging all realizations of the shifts together
merged2 = merged.groupby(['ID']).agg({
    'Language_1': lambda x: ', '.join(x.unique()),
    'Lexemes_1':   lambda x: ', '.join(x.unique()),
    'Meanings_1':  lambda x: ', '.join(x.unique()),
    'Meaning_1': lambda x: ','.join(x.unique()),  
    'Languages_2': lambda x: ', '.join(x.unique()), 
    'Lexemes_2':   lambda x: ', '.join(x.unique()),
    'Meanings_2':  lambda x: ', '.join(x.unique()),
    'Meaning_2': lambda x: ','.join(x.unique()),
    'Type':       'first',   
    'Direction':  'first'
}).reset_index()

merged2

,ID,Language_1,Lexemes_1,Meanings_1,Meaning_1,Languages_2,Lexemes_2,Meanings_2,Meaning_2,Type,Direction
0,shift0007,"Ancient Greek, Irish Gaelic","Ancient Greek: [μετατρέπω], Irish Gaelic: [iom...","μετατρέπω: [to turn back], iompaigh: [to turn,...","to turn back,to turn, rotate (intr.)","Ancient Greek: [Modern Greek], Irish Gaelic: [...","μετατρέπω: [μετατρέπω], iompaigh: [iompaigh]","μετατρέπω: [to transform], iompaigh: [to become]","to transform,to become",Semantic evolution,→
1,shift0008,Irish Gaelic,Irish Gaelic: [eolas],eolas: [to know],to know,Irish Gaelic: [Irish Gaelic],eolas: [eolas],eolas: [to know how],to know how,Semantic evolution,→
2,shift0013,Irish Gaelic,Irish Gaelic: [comhair],"comhair: [to calculate, count]","to calculate, count",Irish Gaelic: [Irish Gaelic],comhair: [comhair],comhair: [to take into account],to take into account,Semantic evolution,→
3,shift0016,Irish Gaelic,Irish Gaelic: [milis],milis: [sweet (taste)],sweet (taste),Irish Gaelic: [Irish Gaelic],milis: [milis],"milis: [dear, darling]","dear, darling",Semantic evolution,→
4,shift0035,"French, Irish Gaelic","French: [considérer], Irish Gaelic: [feic]","considérer: [to stare, to gaze], feic: [to see...","to stare, to gaze,to see/to look at","French: [French], Irish Gaelic: [Irish Gaelic]","considérer: [considérer], feic: [feic]","considérer: [to consider], feic: [to have opin...","to consider,to have opinion",Semantic evolution,→
...,...,...,...,...,...,...,...,...,...,...,...
229,shift4646,Welsh,Welsh: [drych],drych: [to see/to look at],to see/to look at,Welsh: [Welsh],drych: [drych],drych: [mirror],mirror,Semantic evolution,→
230,shift4647,Portuguese,Portuguese: [dona],"dona: [female owner, mistress]","female owner, mistress",Portuguese: [Brazilian Portuguese],dona: [dona],dona: [wife],wife,Semantic evolution,→
231,shift4663,Latin,Latin: [libet (impers.)],libet (impers.): [to want],to want,Latin: [Medieval Latin],libet (impers.): [quodlibet],quodlibet: [anything],anything,Semantic evolution,→
232,shift4720,Old East Slavic,Old East Slavic: [дрѣмѫчии],дрѣмѫчии: [sleeping],sleeping,Old East Slavic: [Old East Slavic],дрѣмѫчии: [дремучий],дремучий: [dense (forest)],dense (forest),Semantic evolution,→


In [8]:
#Rearranging the columns
merged2 = merged2.iloc[:, [0, 9, 1, 2, 3, 4, 10, 5, 6, 7, 8]]
merged2
merged2.to_csv('MergedShifts.csv')

### Sampling the Data

Randomly sampling 200 rows from a dataframe for annotation. 100 of these rows are saved with all the information and the other 100 rows are saved without any language information. The remaining shifts are written to another document.

In [9]:
#To ensure that the results are consistent across trial
random.seed(3)

#Importing the dataframe
data = pd.read_csv('MergedShifts.csv')

#Since this column is the row number, it is unique to each row. So it is used for sampling.
#Putting all row numbers into a list
id_nums = data[['ID']].values.tolist()

#Since after 'tolist' items are put as list into list, this loop puts them as single items into a list
ids = []
for i in id_nums:
    ids.append(i[0])

#Sampling 200 row numbers
sample_id = random.sample(ids, 200)

#Saving the remaining ids 
remaining = list(set(ids) - set(sample_id))

#Sampling 100 row numbers from the sampled row numbers for the rows with all information
id_lang = random.sample(sample_id, 100)

#Removing the 100 sampled row numbers from all of the row numbers
id_nolang = list(set(sample_id) - set(id_lang))

#Saving the first sample of rows
lang = data[data['ID'].isin(id_lang)]

#Remaining shifts
remaining_data = data[data['ID'].isin(remaining)]

#Saving the second sample of rows
nolang = data[data['ID'].isin(id_nolang)]

#Removing the language information from the second sample and the row number column, only keeping the meaning column without lexeme info
nolang = nolang[['ID', 'Language_1', 'Lexemes_1', 'Meanings_1', 'Direction',
       'Languages_2', 'Lexemes_2', 'Meanings_2']]

#Removing the row number column from the first sample and meaning info without the lexeme info columns
lang = lang[['ID', 'Language_1', 'Lexemes_1', 'Meanings_1', 'Direction',
       'Languages_2', 'Lexemes_2', 'Meanings_2']]

#Removing the row number column from the remaining sample and meaning info without the lexeme info columns
remaining_data = remaining_data[['ID', 'Language_1', 'Lexemes_1', 'Meanings_1', 'Direction',
       'Languages_2', 'Lexemes_2', 'Meanings_2']]

#Saving the sample with language information
lang.to_csv('RandomShiftsLang.csv')

#Saving the sample without language information
nolang.to_csv('RandomShifts.csv')

#Saving the remaining shifts
remaining_data.to_csv('RemainingShifts.csv')


Part below is written because we did the sampling of 200 shifts with an earlier version of DatSemShift and then we updated our dataset to the latest version. To separate the remaining shifts from the ones we have already annotated we have written this code. So it is not an essential part of the dataset construction.

In [ ]:
'''#Importing the dataframe
data = pd.read_csv('MergedShifts.csv')

data_lang = pd.read_csv('Annotations/RandomShiftsLang_Ece.csv')

id_lang = list(data_lang['ID'])

data_nolang = pd.read_csv('Annotations/RandomShifts_Ece.csv')

id_nolang = list(data_nolang['ID'])

id_nums = data[['ID']].values.tolist()

ids = []
for i in id_nums:
    ids.append(i[0])

remaining = list(set(ids) - set(id_lang) - set(id_nolang))

remaining_data = data[data['ID'].isin(remaining)]

remaining_data = remaining_data[['ID', 'Language_1', 'Lexemes_1', 'Meanings_1', 'Direction',
       'Languages_2', 'Lexemes_2', 'Meanings_2']]

remaining_data.to_csv('RemainingShifts.csv')'''